<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/File_Utilities_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%writefile FileUtilities.py
# 08/09/2020-1
from __future__ import absolute_import
from IPython.display import clear_output
import glob, os, pathlib, sys
from os.path import *
from pathlib import Path
from google.colab import drive, files

global C
global fu

class ColorOutput:
    LITE = '\033[96m'; HEADER = '\033[95m'; BLUE = '\033[94m';
    WARNING = '\033[93m'; GREEN = '\033[92m'; RED = '\033[91m'; 
    DARK = '\033[90m'; UNDERLINE = '\033[4m';BOLD = '\033[1m'; 
    WHITE = '\033[0m'

C = ColorOutput()

class FileUtilities(ColorOutput):
    __all__ = ['C', 'allFiles', 'allFilesDict', 'cwdPath', 'homePath', 
           'inspectFile', 'listFiles', 'printSystemPaths', 
           'setSystemPaths', 'updateAllFiles', 'mountGoogleDrive']

    def __init__(self):
        self.C = ColorOutput
        self.homePath = Path.home()
        self.cwdPath = Path.cwd()
        self.allFiles = glob.glob('**', recursive=True)
        self.allFilesDict = {}

    def updateAllFiles(self):
        self.allFiles = glob.glob('**', recursive=True)

    def setSystemPaths(self, addNewPath=None):
        addNewPath = realpath(addNewPath)
        pathList = sys.path.copy()
        if not addNewPath in pathList and exists(addNewPath) and addNewPath != 'None':
            sys.path.append(addNewPath)

    def listFiles(self, silent=False):
        self.updateAllFiles()
        self.allFiles = sorted(self.allFiles)
        tempDict = self.allFilesDict
        fileCount = 0
        if not silent:
            print(f'{C.BLUE}{self.cwdPath}')
        self.setSystemPaths(addNewPath=self.cwdPath)
        for fil in self.allFiles:
            fullPath = Path(realpath(fil)) # POSIX path
            realPath = realpath(fil) # String path
            if isdir(fullPath) and not realPath.__contains__('__pycache__'):  
                if not silent:
                    print(f'\n{C.BLUE}{fullPath}')
                self.setSystemPaths(addNewPath=realPath) 
            elif isfile(fullPath) and not realPath.__contains__('__pycache__'): 
                fileCount += 1
                if not silent:
                    print(f'{C.WHITE}{fileCount}. {fullPath.name}') 
                tempDict[fileCount] = fullPath
                self.allFilesDict = {}
                self.allFilesDict.update(tempDict)

    def inspectFile(self, selection):
        if selection <= len(self.allFilesDict):  
            try:
                pth = self.allFilesDict[selection]  
                with open(pth, 'r') as openFile:  
                    print(f'# %%writefile {pth.name}\n')  
                    print(openFile.read())
            except:
                print(f'{C.RED}BULLSHIT')

    def printSystemPaths(self):
        pathList = sys.path.copy() 
        pathCount = 0
        for pth in sorted(pathList):
            pathCount += 1
            print(f'{C.WHITE}{pathCount}. {C.BLUE}{pth}')

    def mountGoogleDrive(self):
        drive.mount('gdrive', force_remount=True, timeout_ms=600000)

fu = FileUtilities()
fu.mountGoogleDrive()
fu.listFiles(silent=True)

In [ ]:
print(f'home: {C.GREEN}{fu.homePath}{C.WHITE}')
print(f'cwd:  {C.GREEN}{fu.cwdPath}{C.WHITE}')
%ls

In [ ]:
fu.listFiles(silent=False)

In [ ]:
pth = fu.allFilesDict[1]
print(pth)
pth = fu.allFilesDict[6]
print(pth)

!cp {fu.allFilesDict[1]} "/content/gdrive/My Drive/Colab Notebooks"
# with open(pth, 'r') as openFile:
    # print(openFile.read())

In [ ]:
fu.inspectFile(selection=1)

In [ ]:
deleteFilesDict = {}
def searchFiles():
    fileCounter = 0
    fu.updateAllFiles()
    for fil in sorted(fu.allFiles):
        if fil.__contains__('('):
            fileCounter += 1
            deleteFilesDict[fileCounter] = fil
            print(f'{fileCounter} .{fil}')

searchFiles()

def deleteFile(filePath=pth):
    if exists(filePath) and isfile(filePath):
        clear_output()
        print(f'{C.RED}{filePath}{C.WHITE}\n')
        os.remove(filePath)
        searchFiles()
try:
    pth = deleteFilesDict[1]
    deleteFile(filePath=pth)
except Exception as err:
    print(err)